In [1]:
from typing import Tuple
from typing import List
import numpy as np
import pathlib
import json

In [2]:
class Wanderer:
    '''
    '''
    def __init__(self, name: str, m: float, x0: float, y0: float, vx0: float, vy0: float) -> None:
        self.xn, self.yn, self.vxn, self.vyn = list(), list(), list(), list()
        self.m: float = m
        self.name: str = name

        self.xn.append(x0)
        self.yn.append(y0)
        self.vxn.append(vx0)
        self.vyn.append(vy0)

        self.jbodies = list()



In [3]:
class NBodies:
    '''
    '''
    def __init__(self, wanderers: Tuple[Wanderer], t: int, h: float) -> None:
        self.aobjects = dict()

        self.G = G = 880 * 10**-12
        self.t = t
        self.h = h
        for aobject in wanderers:
            aobject.m = aobject.m * self.G  # modifying mass to include gravity

            self.aobjects[aobject.name] = aobject

        for aobject in wanderers:
            for jobject in wanderers:
                if jobject.name not in aobject.name:
                    aobject.jbodies.append(jobject)


    def rk4_f(self, r, k1, k2, k3, k4) -> float:
        '''
        Probably could be a lambda function, but there is something nice about seeing this explicetly written out.
        '''

        return r + (self.h / 6) * (k1 + (2*k2) + (2*k3) + k4)


    def acc(self, xy: tuple, Janders: list, step) -> Tuple[float, float]:
        '''
        '''
        ai = [0, 0]

        for J in Janders:
            r = (xy[0] - J.xn[step])**2 + (xy[1] - J.yn[step])**2

            a = J.m / r

            ai[0] += a * (xy[0] - J.xn[step]) / np.sqrt(r)
            ai[1] += a * (xy[1] - J.yn[step]) / np.sqrt(r)

        return ai



    def vel(self, vxy: np.array, eta_xy: np.array):
        '''
        '''

        return vxy[0] + (eta_xy[0]*self.h), vxy[1] + (eta_xy[1]*self.h)
    


    def rk4_step(self, step, *args, **kwargs) -> Tuple[float, float]:
        '''
        '''
        for body in self.aobjects.values():
            xy = (body.xn[step], body.yn[step])
            vxy = (body.vxn[step], body.vyn[step])

            m1 = self.vel(vxy, (0, 0))
            n1 = self.acc(xy, body.jbodies, step)

            m2 = self.vel(vxy, n1)
            n2 = self.acc((xy[0] + (m1[0] * self.h / 2), xy[1] + (m1[1] * self.h / 2)), body.jbodies, step)

            m3 = self.vel(vxy, n2)
            n3 = self.acc((xy[0] + (m2[0] * self.h / 2), xy[1] + (m2[1] * self.h / 2)), body.jbodies, step)

            m4 = self.vel(vxy, n3)
            n4 = self.acc((xy[0] + (m3[0] * self.h / 2), xy[1] + (m3[1] * self.h / 2)), body.jbodies, step)

            body.xn.append(self.rk4_f(xy[0], m1[0], m2[0], m3[0], m4[0]))
            body.yn.append(self.rk4_f(xy[1], m1[1], m2[1], m3[1], m4[1]))
            body.vxn.append(self.rk4_f(vxy[0], n1[0], n2[0], n3[0], n4[0]))
            body.vyn.append(self.rk4_f(vxy[1], n1[1], n2[1], n3[1], n4[1]))


    
    def oribts(self):
        '''
        '''

        for step in range(self.t):
            self.rk4_step(step)


In [4]:
cwd = pathlib.Path.cwd()
ao_json_path = cwd / "AstronomicalObjects.json"

with open(ao_json_path, 'r') as ao_file:
    ao_file = ao_file.read()
ao_json = json.loads(ao_file)['System']

system = list()
for body in ao_json.values():
    w = Wanderer(body["name"], 
        float(body["m"]), 
        float(body["x0"]), 
        float(body["y0"]), 
        float(body["vx0"]), 
        float(body["vy0"]))
    
    system.append(w)
    
system = tuple(system)
# print(system[0].m)
for o in system:
    print(o.m)
    

30000.0
1.0


In [5]:
# print(len(system[0].xn))
nbodies = NBodies(system, t = 1825, h = 1)
nbodies.oribts()

1


In [6]:
# print(len(system[0].xn))
# have this output to a csv file or to plotly

1826
